In [304]:
import zipfile  
import os
import pandas as pd
import numpy as np
import statistics

import tensorflow as tf
from tensorflow.keras.layers import Dense, Softmax, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
z = zipfile.ZipFile('/content/drive/MyDrive/AIU/Test task moto races/dataset_sample.zip')


In [ ]:
z.extractall('/content/drive/MyDrive/AIU/Test task moto races')

In [ ]:
dataSet = pd.read_csv('/content/drive/MyDrive/AIU/Test task moto races/dataset_sample/race_2008_2015.csv')

In [310]:
# lets check how does predict file looks like

predictCSV = pd.read_csv('/content/drive/MyDrive/AIU/Test task moto races/dataset_sample/predict.csv')
predictCSV.head()

,raceinfo_id,place_num,place_code,raceno,distance,situation,raceclass_code,weather_name3,popular,roadtemp,temp,humid,motorbikeno,player_code,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,rank_now,rank_last,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,last90_numall,last90_numwin,last90_numwinwin,last90_hightime,last90_highplace,last90_highplace_home,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
0,52674,2,kawaguchi,1,3100,good,3,sunny,2,14,9,39,1,1315,0,3.46,1,0.066,3.526,3.526,64,13,B-103,B-108,43.985,0,0,0,10,0.0,0.0,3.47,3.536,3.500,22,0,0,3.500,kawaguchi,1,...,0.0,3.539,3.47,0.16,3.493,3.493,3.50,3.556,5,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.17,3.516,3.516,3.49,3.550,3,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.19,3.520,3.520
1,52674,2,kawaguchi,1,3100,good,3,sunny,2,14,9,39,2,2119,0,3.42,0,0.097,3.517,3.517,48,21,B-101,B-88,45.820,0,1,0,9,10.0,10.0,3.43,3.527,3.506,20,0,0,3.506,kawaguchi,0,...,0.0,3.561,3.44,0.17,3.507,3.507,3.47,3.563,2,0,0,0,0,0,0.0,0.0,0.0,0.0,3.561,3.46,0.19,3.513,3.513,3.43,3.527,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.517,3.517
2,52674,2,kawaguchi,1,3100,good,3,sunny,2,14,9,39,3,2005,10,3.41,0,0.086,3.496,3.506,53,20,B-82,B-86,49.582,0,1,1,8,10.0,20.0,3.42,3.506,3.474,21,0,0,3.474,iizuka,1,...,0.0,3.500,3.45,0.18,3.440,3.450,3.50,3.520,3,1,0,0,0,0,33.3,33.3,33.3,0.0,3.500,3.45,0.20,3.430,3.440,3.42,3.506,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.496,3.506
3,52674,2,kawaguchi,1,3100,good,3,sunny,2,14,9,39,4,1214,10,3.36,1,0.095,3.455,3.465,63,12,B-51,B-55,53.643,0,2,2,6,20.0,40.0,3.40,3.495,3.481,25,0,0,3.477,kawaguchi,1,...,0.0,3.494,3.40,0.20,3.456,3.466,3.44,3.504,4,0,2,0,0,0,0.0,50.0,50.0,0.0,3.494,3.40,0.17,3.424,3.434,3.44,3.504,4,0,2,0,0,0,0.0,50.0,50.0,0.0,3.494,3.40,0.17,3.424,3.434
4,52674,2,kawaguchi,1,3100,good,3,sunny,2,14,9,39,5,2514,20,3.35,1,0.097,3.447,3.467,42,25,A-227,B-37,59.579,1,2,2,5,30.0,50.0,3.38,3.477,3.460,28,0,0,3.458,hamamatsu,0,...,11.1,3.465,3.39,0.18,3.384,3.404,3.53,3.529,5,0,0,1,0,0,0.0,0.0,20.0,20.0,3.465,3.39,0.22,3.349,3.369,3.50,3.552,3,0,0,0,0,0,0.0,0.0,0.0,33.3,3.543,3.45,0.25,3.402,3.422


In [ ]:
# quick overview of the dataset and check for extremes. We will come back here later, if accuracy will not be sufficient
dataSet.describe()

,raceinfo_id,place_num,raceno,distance,raceclass_code,popular,roadtemp,temp,humid,motorbikeno,player_code,xorder,racetime,all_racetime,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,last90_numall,last90_numwin,last90_numwinwin,last90_hightime,last90_highplace_home,last90_stave,last180_rate0_rate2th,last180_rate0_numall,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
count,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,302488.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,...,303149.000000,303149.000000,303149.000000,303146.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.00000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303148.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303149.000000,303146.000000,303149.000000,303149.000000
mean,30695.001056,3.150279,6.672887,3112.640649,2.925090,2.144668,30.361190,20.488126,52.530026,4.495387,2218.231965,4.511432,3.494139,109.176797,19.314067,3.441009,0.503403,0.089275,3.528876,3.548190,42.277966,21.745221,73.059069,1.265440,1.251985,1.248036,5.992885,25.488494,38.110572,3.394554,3.483990,3.451477,25.071110,0.618930,0.082257,3.445832,0.567605,0.157664,25.403414,33.537676,...,1.073310,3.440367,3.357003,0.156803,3.514781,3.534095,3.429069,3.502788,6.011140,0.849279,0.798746,0.779227,0.143533,0.02157,12.521662,24.744318,37.089720,1.004436,3.350666,3.271219,0.151768,3.514729,3.534043,3.416491,3.495595,2.257837,0.323933,0.304527,0.293753,0.047056,0.007528,8.222977,16.162520,23.862987,0.568381,2.085888,2.039326,0.094464,3.520113,3.539427
std,13383.484116,1.532991,3.425647,120.633740,2.305032,1.486108,13.351625,8.394516,18.932398,2.289388,822.944995,2.316131,0.383361,13.555980,15.807984,0.179535,0.499989,0.013966,0.195490,0.195575,11.302591,6.734417,20.713305,1.295886,1.112466,1.086386,1.991308,17.450851,19.707839,0.128473,0.131238,0.130200,6.517717,1.169038,0.360169,0.098774,0.495409,0.037443,13.657862,8.724031,...,4.126659,0.242427,0.237192,0.042194,0.180875,0.18092

In [ ]:
#raceclass_code = 0.0; temperature of the race = 72.0; xorder = 9; racetime = 0; all_racetime = 0; hensa = 0; shisou_hensa = 0; soutei_time = 0; age = 74.0; all p2>100;

In [ ]:
# percentages of nan values in columns. Quite clean DF.
dataSet[dataSet.columns[dataSet.isnull().any()]].isnull().sum() *100 / dataSet.shape[0]

weather_name3                 0.002639
racetime                      0.218045
rank_last                     0.270494
last90_highplace              0.064325
p2_0_180_All_All_stave        0.000990
p2_0_90_All_All_stave         0.000330
p2_1_90_All_Here_stave        0.000330
p2_All_180_All_All_stave      0.000990
p2_All_90_All_All_stave       0.000330
p2_All_90_All_Here_stave      0.000330
p2_All_90_10_All_stave        0.000330
p2_All_90_10_Here_stave       0.000660
p2_All_Life_All_All_stave     0.023421
p2_Other_180_All_All_stave    0.000990
p2_Other_90_All_All_stave     0.000330
p2_Other_90_All_Here_stave    0.000990
dtype: float64

In [ ]:
# Due to relatively small amt of nan values in the DF, we can substitute these values with averages for the column, assuming
# low interference to the data structure itself

dataSet.fillna(dataSet.mean(), inplace = True) 
dataSet['weather_name3'].fillna('sunny', inplace = True)
dataSet['last90_highplace'].fillna(0.0, inplace = True)

In [ ]:
# now we have a look at the picture again.
dataSet[dataSet.columns[dataSet.isnull().any()]].isnull().sum() *100 / dataSet.shape[0]

rank_last    0.270494
dtype: float64

In [ ]:
# getting rid of races with number of drivers =! 8

def replaceNull(dataset) : 
 
    #dataset_train.isnull().sum()
    #dataset_test.isnull().sum()

    print('Before removing absent or accident players')
    print(dataset.shape)
    
    # remove absent or accident players. 
    dataset = dataset[ (dataset['xorder'] >= 1) & (dataset['xorder'] <= 8) ]

    print('After removing absent or accident players ')
    print(dataset.shape)

    return dataset

dataSetn = replaceNull(dataSet)

# group by raceinfo_id
group_by_raceInfo = dataSetn.groupby('raceinfo_id')['all_race_order']

# find race nrs, that do not have 8 participants

not8driversRaceNrlist=[]
for key, value in group_by_raceInfo.groups.items():
  if len(value)!=8:
    not8driversRaceNrlist.append(key)

# Now lets identify indices, that we need to get rid off
remIdxLst = []
for i in range(len(not8driversRaceNrlist)):
  remIdxLst.extend(dataSetn.index[dataSetn['raceinfo_id']==not8driversRaceNrlist[i]])

# drop indices
dataSetn.drop(index=remIdxLst, inplace=True)

In [ ]:
dataSetn.to_csv('/content/drive/MyDrive/AIU/Test task moto races/dataPrep.csv')

In [2]:
# dset with all the rows that we will use

dataSetn = pd.read_csv('/content/drive/MyDrive/AIU/Test task moto races/dataPrep.csv')
dataSetn.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
dataSetn.head()

,raceinfo_id,hold_date,place_num,place_code,raceno,distance,situation,raceclass_code,weather_name3,popular,roadtemp,temp,humid,motorbikeno,player_code,all_refund,all_race_order,xorder,racetime,all_racetime,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,rank_now,rank_last,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
0,7321,2008/1/2 0:00,2,kawaguchi,1,3100,good,1,sunny,4,19.0,9,42,1,602,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,3,3.409,106.7017,30,3.32,1,0.088,3.408,3.438,59,6,A-37,S-38,81.724,0,3,3,3,30.0,60.0,3.34,3.428,3.410,...,0.0,3.397,3.34,0.18,3.430,3.460,3.34,3.428,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.408,3.438,3.34,3.428,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.408,3.438
1,7321,2008/1/2 0:00,2,kawaguchi,1,3100,good,1,sunny,4,19.0,9,42,2,2620,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,2,3.403,106.5139,30,3.29,1,0.099,3.389,3.419,32,26,S-52,S-50,89.141,0,1,2,6,10.0,30.0,3.33,3.429,3.383,...,0.0,3.395,3.30,0.17,3.322,3.352,3.45,3.498,8,0,0,2,2,0,0.0,0.0,25.0,0.0,3.395,3.30,0.18,3.338,3.368,3.43,3.489,2,0,0,1,1,0,0.0,0.0,50.0,0.0,3.422,3.38,0.14,3.349,3.379
2,7321,2008/1/2 0:00,2,kawaguchi,1,3100,good,1,sunny,4,19.0,9,42,3,2425,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,6,3.452,107.7024,20,3.36,1,0.083,3.443,3.463,35,24,A-90,A-72,75.002,1,0,0,8,10.0,10.0,3.36,3.443,3.428,...,50.0,3.447,3.43,0.20,3.377,3.397,3.36,3.443,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.443,3.463,3.36,3.443,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.443,3.463
3,7321,2008/1/2 0:00,2,kawaguchi,1,3100,good,1,sunny,4,19.0,9,42,4,2625,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,5,3.436,107.2032,20,3.35,1,0.071,3.421,3.441,33,26,A-18,A-87,84.382,1,0,2,7,10.0,30.0,3.35,3.421,3.397,...,0.0,3.395,3.34,0.17,3.402,3.422,3.45,3.492,4,0,0,1,1,0,0.0,0.0,25.0,0.0,3.429,3.37,0.18,3.392,3.412,3.52,3.540,2,0,0,0,1,0,0.0,0.0,0.0,0.0,3.449,3.41,0.12,3.370,3.390
4,7321,2008/1/2 0:00,2,kawaguchi,1,3100,good,1,sunny,4,19.0,9,42,5,2912,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,8,3.502,108.9122,10,3.36,1,0.084,3.444,3.454,29,29,A-172,B-1,64.817,1,1,1,6,22.2,33.3,3.38,3.464,3.441,...,0.0,3.436,3.37,0.18,3.341,3.351,3.39,3.436,1,0,0,0,1,0,0.0,0.0,0.0,0.0,3.436,3.39,0.16,3.406,3.416,3.38,3.464,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.444,3.454


In [ ]:
for i,el in enumerate(dataSetn.columns[0:30]):
  print(i, el) 

////////////////////////////////////////
# Data prepration

In [3]:
# function to convert string rank to digit. No column removal/change places

def rank(x):

  if isinstance(x, float) :
      return 300;
  
  alpha = x[:1]
  num = int(x[2:])
  if alpha == 'S' :
      num = num
  elif alpha == 'A' :
      num += 100
  elif alpha == 'B' :
      num += 300
  elif alpha == 'C' :
      num += 400
  return num

def weather(x):

  weather = ['sunny', 'cloud', 'rain', 'lightrain', 'lightsnow', 'snow']
  out = 0

  if x == weather[0]:
    out = 1
  elif x == weather[1]:
    out = 2
  elif x == weather[2]:
    out = 3
  elif x == weather[3]:
    out = 4
  elif weather == weather[4]:
    out = 5
  else:
    out = 6
  
  return out

def situation(x):

  situation = ['good', 'rough', 'wind', 'wet', 'rash', 'oil']
  out = 0

  if x == situation[0]:
    out = 1
  elif x == situation[1]:
    out = 2
  elif x == situation[2]:
    out = 3
  elif x == situation[3]:
    out = 4
  elif x == situation[4]:
    out = 5
  else:
    out = 6
  
  return out

def distance(x):

  distance = [3100, 4100, 5100, 3600]
  out = 0
  
  if x == distance[0]:
    out = 1
  elif x == distance[1]:
    out = 2
  elif x == distance[2]:
    out = 3
  else:
    out = 4

  return out

def age(x):

  out = 0
  if (x > 0) and (x <= 21):
    out = 1
  elif (x > 21) and (x <= 26):
    out = 2
  elif (x > 26) and (x <= 31):
    out = 3
  elif (x > 31) and (x <= 36):
    out = 4
  elif (x > 36) and (x <= 41):
    out = 5    
  elif (x > 41) and (x <= 46):
    out = 6
  elif (x > 46) and (x <= 51):
    out = 7
  else:
    out = 8
  
  return out

def location(x):
  
  plcs = ['kawaguchi', 'funabashi', 'sanyou', 'hamamatsu', 'iizuka', 'isesaki', '0.0']
  out = 0

  if x == plcs[0]:
    out = 0
  elif x == plcs[1]:
    out = 1
  elif x == plcs[2]:
    out = 2
  elif x == plcs[3]:
    out = 3
  elif x == plcs[4]:
    out = 4
  elif x == plcs[5]:
    out = 5
  else:
    out = 6
  
  return out

def bets(x):

  refund = x.split(':')

  Quinella_refund = float(refund[3])
  Exacta_refund = float(refund[5])
  Trio_refund = float(refund[6])
  Trifecta_refund = float(refund[7])

  return Quinella_refund, Exacta_refund, Trio_refund, Trifecta_refund


In [307]:
def convert_data (dataset) :
    
    dataset = dataset.copy()

    dataset['rank_now'] = dataset['rank_now'].apply(rank)
    dataset['rank_last'] = dataset['rank_last'].apply(rank)
    dataset['weather_name3'] = dataset['weather_name3'].apply(weather)
    dataset['situation'] = dataset['situation'].apply(situation)
    dataset['age'] = dataset['age'].apply(age)
    dataset['distance'] = dataset['distance'].apply(distance)
    dataset['place_code'] = dataset['place_code'].apply(location)
    dataset['last90_highplace'] = dataset['last90_highplace'].apply(location)  

    return dataset

In [308]:
# prepare all dfs

df = convert_data(dataSetn)
dfy = dataSetn['xorder'].values
dfBets = pd.DataFrame()
dfBets['Quinella_refund', 'Exacta_refund', 'Trio_refund', 'Trifecta_refund'] = dataSetn['all_refund'].apply(bets)

In [311]:
# now lets drop columns, that we are not planning to use and so that it would mirror the predict table

# lets check how does predict file looks like

predictCSV = convert_data(predictCSV)
predictCSV.head()

,raceinfo_id,place_num,place_code,raceno,distance,situation,raceclass_code,weather_name3,popular,roadtemp,temp,humid,motorbikeno,player_code,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,rank_now,rank_last,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,last90_numall,last90_numwin,last90_numwinwin,last90_hightime,last90_highplace,last90_highplace_home,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
0,52674,2,0,1,1,1,3,1,2,14,9,39,1,1315,0,3.46,1,0.066,3.526,3.526,8,13,403,408,43.985,0,0,0,10,0.0,0.0,3.47,3.536,3.500,22,0,0,3.500,0,1,...,0.0,3.539,3.47,0.16,3.493,3.493,3.50,3.556,5,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.17,3.516,3.516,3.49,3.550,3,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.19,3.520,3.520
1,52674,2,0,1,1,1,3,1,2,14,9,39,2,2119,0,3.42,0,0.097,3.517,3.517,7,21,401,388,45.820,0,1,0,9,10.0,10.0,3.43,3.527,3.506,20,0,0,3.506,0,0,...,0.0,3.561,3.44,0.17,3.507,3.507,3.47,3.563,2,0,0,0,0,0,0.0,0.0,0.0,0.0,3.561,3.46,0.19,3.513,3.513,3.43,3.527,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.517,3.517
2,52674,2,0,1,1,1,3,1,2,14,9,39,3,2005,10,3.41,0,0.086,3.496,3.506,8,20,382,386,49.582,0,1,1,8,10.0,20.0,3.42,3.506,3.474,21,0,0,3.474,4,1,...,0.0,3.500,3.45,0.18,3.440,3.450,3.50,3.520,3,1,0,0,0,0,33.3,33.3,33.3,0.0,3.500,3.45,0.20,3.430,3.440,3.42,3.506,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.496,3.506
3,52674,2,0,1,1,1,3,1,2,14,9,39,4,1214,10,3.36,1,0.095,3.455,3.465,8,12,351,355,53.643,0,2,2,6,20.0,40.0,3.40,3.495,3.481,25,0,0,3.477,0,1,...,0.0,3.494,3.40,0.20,3.456,3.466,3.44,3.504,4,0,2,0,0,0,0.0,50.0,50.0,0.0,3.494,3.40,0.17,3.424,3.434,3.44,3.504,4,0,2,0,0,0,0.0,50.0,50.0,0.0,3.494,3.40,0.17,3.424,3.434
4,52674,2,0,1,1,1,3,1,2,14,9,39,5,2514,20,3.35,1,0.097,3.447,3.467,6,25,327,337,59.579,1,2,2,5,30.0,50.0,3.38,3.477,3.460,28,0,0,3.458,3,0,...,11.1,3.465,3.39,0.18,3.384,3.404,3.53,3.529,5,0,0,1,0,0,0.0,0.0,20.0,20.0,3.465,3.39,0.22,3.349,3.369,3.50,3.552,3,0,0,0,0,0,0.0,0.0,0.0,33.3,3.543,3.45,0.25,3.402,3.422


In [312]:
df.head() # delete 6 columns from xTrain - hold_date, all_refund, all_race_order, xorder, racetime, all_racetime, 

,raceinfo_id,hold_date,place_num,place_code,raceno,distance,situation,raceclass_code,weather_name3,popular,roadtemp,temp,humid,motorbikeno,player_code,all_refund,all_race_order,xorder,racetime,all_racetime,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,rank_now,rank_last,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
0,7321,2008/1/2 0:00,2,0,1,1,1,1,1,4,19.0,9,42,1,602,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,3,3.409,106.7017,30,3.32,1,0.088,3.408,3.438,8,6,137,38,81.724,0,3,3,3,30.0,60.0,3.34,3.428,3.410,...,0.0,3.397,3.34,0.18,3.430,3.460,3.34,3.428,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.408,3.438,3.34,3.428,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.408,3.438
1,7321,2008/1/2 0:00,2,0,1,1,1,1,1,4,19.0,9,42,2,2620,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,2,3.403,106.5139,30,3.29,1,0.099,3.389,3.419,4,26,52,50,89.141,0,1,2,6,10.0,30.0,3.33,3.429,3.383,...,0.0,3.395,3.30,0.17,3.322,3.352,3.45,3.498,8,0,0,2,2,0,0.0,0.0,25.0,0.0,3.395,3.30,0.18,3.338,3.368,3.43,3.489,2,0,0,1,1,0,0.0,0.0,50.0,0.0,3.422,3.38,0.14,3.349,3.379
2,7321,2008/1/2 0:00,2,0,1,1,1,1,1,4,19.0,9,42,3,2425,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,6,3.452,107.7024,20,3.36,1,0.083,3.443,3.463,4,24,190,172,75.002,1,0,0,8,10.0,10.0,3.36,3.443,3.428,...,50.0,3.447,3.43,0.20,3.377,3.397,3.36,3.443,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.443,3.463,3.36,3.443,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.443,3.463
3,7321,2008/1/2 0:00,2,0,1,1,1,1,1,4,19.0,9,42,4,2625,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,5,3.436,107.2032,20,3.35,1,0.071,3.421,3.441,4,26,118,187,84.382,1,0,2,7,10.0,30.0,3.35,3.421,3.397,...,0.0,3.395,3.34,0.17,3.402,3.422,3.45,3.492,4,0,0,1,1,0,0.0,0.0,25.0,0.0,3.429,3.37,0.18,3.392,3.412,3.52,3.540,2,0,0,0,1,0,0.0,0.0,0.0,0.0,3.449,3.41,0.12,3.370,3.390
4,7321,2008/1/2 0:00,2,0,1,1,1,1,1,4,19.0,9,42,5,2912,900:110_100_100:0:740:250_220_120:2310:320:3940,7_2_1_8_4_3_6_5,8,3.502,108.9122,10,3.36,1,0.084,3.444,3.454,3,29,272,301,64.817,1,1,1,6,22.2,33.3,3.38,3.464,3.441,...,0.0,3.436,3.37,0.18,3.341,3.351,3.39,3.436,1,0,0,0,1,0,0.0,0.0,0.0,0.0,3.436,3.39,0.16,3.406,3.416,3.38,3.464,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.444,3.454


In [313]:
dfx = df.drop(columns=['hold_date', 'all_refund', 'all_race_order', 'xorder', 'racetime', 'all_racetime'])

In [314]:
# final checkout
dfx.head()

,raceinfo_id,place_num,place_code,raceno,distance,situation,raceclass_code,weather_name3,popular,roadtemp,temp,humid,motorbikeno,player_code,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,rank_now,rank_last,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,last90_numall,last90_numwin,last90_numwinwin,last90_hightime,last90_highplace,last90_highplace_home,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
0,7321,2,0,1,1,1,1,1,4,19.0,9,42,1,602,30,3.32,1,0.088,3.408,3.438,8,6,137,38,81.724,0,3,3,3,30.0,60.0,3.34,3.428,3.410,21,1,0,3.410,0,1,...,0.0,3.397,3.34,0.18,3.430,3.460,3.34,3.428,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.408,3.438,3.34,3.428,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.408,3.438
1,7321,2,0,1,1,1,1,1,4,19.0,9,42,2,2620,30,3.29,1,0.099,3.389,3.419,4,26,52,50,89.141,0,1,2,6,10.0,30.0,3.33,3.429,3.383,24,2,0,3.383,0,1,...,0.0,3.395,3.30,0.17,3.322,3.352,3.45,3.498,8,0,0,2,2,0,0.0,0.0,25.0,0.0,3.395,3.30,0.18,3.338,3.368,3.43,3.489,2,0,0,1,1,0,0.0,0.0,50.0,0.0,3.422,3.38,0.14,3.349,3.379
2,7321,2,0,1,1,1,1,1,4,19.0,9,42,3,2425,20,3.36,1,0.083,3.443,3.463,4,24,190,172,75.002,1,0,0,8,10.0,10.0,3.36,3.443,3.428,13,1,0,3.428,0,1,...,50.0,3.447,3.43,0.20,3.377,3.397,3.36,3.443,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.443,3.463,3.36,3.443,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.443,3.463
3,7321,2,0,1,1,1,1,1,4,19.0,9,42,4,2625,20,3.35,1,0.071,3.421,3.441,4,26,118,187,84.382,1,0,2,7,10.0,30.0,3.35,3.421,3.397,24,2,0,3.397,0,1,...,0.0,3.395,3.34,0.17,3.402,3.422,3.45,3.492,4,0,0,1,1,0,0.0,0.0,25.0,0.0,3.429,3.37,0.18,3.392,3.412,3.52,3.540,2,0,0,0,1,0,0.0,0.0,0.0,0.0,3.449,3.41,0.12,3.370,3.390
4,7321,2,0,1,1,1,1,1,4,19.0,9,42,5,2912,10,3.36,1,0.084,3.444,3.454,3,29,272,301,64.817,1,1,1,6,22.2,33.3,3.38,3.464,3.441,15,1,0,3.436,1,0,...,0.0,3.436,3.37,0.18,3.341,3.351,3.39,3.436,1,0,0,0,1,0,0.0,0.0,0.0,0.0,3.436,3.39,0.16,3.406,3.416,3.38,3.464,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.444,3.454


/////////////////////////////////
# From pandas to numpy

In [ ]:
group_by_data = df.groupby('raceinfo_id')['raceinfo_id'].mean()

In [ ]:
col_names = df.columns.to_list()

In [ ]:
# create work piece for yTrain

dfy = dataSetn.iloc[:][['raceinfo_id','xorder']]

In [ ]:
# final data acquisition (not time-effective)
result_dfx = pd.DataFrame(columns = col_names)
result_dfy = pd.DataFrame(columns = ['xorder'])

for el in group_by_data:
  row_df = dfx[dfx['raceinfo_id'] == el]

  tmp_x = pd.Series( [row_df[col_names[i]].values for i in range(len(col_names))],
                      index = result_dfx.columns )             
                         
  
  result_dfx = result_dfx.append( tmp_x, ignore_index = True )

  row_dfy = dfy[dfy['raceinfo_id'] == el]

  tmp_y = pd.Series([row_dfy['xorder'].values], index=result_dfy.columns)

  result_dfy = result_dfy.append(tmp_y, ignore_index=True)

In [ ]:
x = result_dfx.values
y = result_dfy.values
np.save('/content/drive/MyDrive/AIU/Test task moto races/x.npy', x)
np.save('/content/drive/MyDrive/AIU/Test task moto races/y.npy', y)

In [ ]:
nx = []
for i in range(x.shape[0]):
  nx.append(np.concatenate((x[i][0],x[i][1],x[i][2])).astype('float32'))


In [ ]:
ny = []
for i in range(y.shape[0]):
  ny.append(y[i][0].astype('float32'))


# Second option of converting pandas to numpy

In [72]:
prep_bets = dfBets.to_numpy()
nyBets = []


for i in range(0,len(prep_bets)):
  if i%8==0:
    tmp = list(map(int,prep_bets[i][0]))
    nyBets.append(tmp)


In [315]:
prep_dfx = dfx.to_numpy(dtype = 'float32')
nx = []
ny = []

tmpX = prep_dfx[0].tolist()
tmpY = [dfy[0]]

for i in range(1,len(prep_dfx)):
  if i%8!=0:
    tmpX.extend(prep_dfx[i])
    tmpY.extend([dfy[i]])
  else:
    nx.append(tmpX)
    ny.append(tmpY)
    tmpX = []
    tmpY = []
    tmpX.extend(prep_dfx[i])
    tmpY.extend([dfy[i]])
nx.append(tmpX)
ny.append(tmpY)

In [316]:
#np.save('/content/drive/MyDrive/AIU/Test task moto races/nx.npy', nx)
#np.save('/content/drive/MyDrive/AIU/Test task moto races/ny.npy', ny)
#np.save('/content/drive/MyDrive/AIU/Test task moto races/nyBets.npy', nyBets)

In [28]:
nx = np.load('/content/drive/MyDrive/AIU/Test task moto races/nx.npy')
ny = np.load('/content/drive/MyDrive/AIU/Test task moto races/ny.npy')
nyBets = np.load('/content/drive/MyDrive/AIU/Test task moto races/nyBets.npy')

In [317]:
#xTrain, xTest, yTrain, yTest = train_test_split(nx,ny, test_size = 0.1)

xTrain = nx[:32000]
xTest = nx[32000:]
yTrainOrd = ny[:32000]
yTestOrd = ny[32000:]
yTrainBet = nyBets[:32000]
yTestBet = nyBets[32000:]

In [318]:
xTrain = np.array(xTrain)
xTest = np.array(xTest)
yTrainOrd = np.array(yTrainOrd)
yTestOrd = np.array(yTestOrd)
yTrainBet = np.array(yTrainBet)
yTestBet = np.array(yTestBet)

In [319]:
print(xTrain.shape, xTest.shape, yTrainOrd.shape, yTestOrd.shape, yTrainBet.shape, yTestBet.shape)

(32000, 2864) (2489, 2864) (32000, 8) (2489, 8) (32000, 4) (2489, 4)


In [320]:
# try both options (slightly better results with StandardScaler)

scalerX = StandardScaler()
scalerX.fit(xTrain)
xTrain = scalerX.transform(xTrain)
xTest = scalerX.transform(xTest)

In [103]:
scalerX = MinMaxScaler()
scalerX.fit(xTrain)
xTrain = scalerX.transform(xTrain)
xTest = scalerX.transform(xTest)

In [321]:
scalerY = MinMaxScaler()
scalerY.fit(yTrainOrd)
yTrain = scalerY.transform(yTrainOrd)
yTest = scalerY.transform(yTestOrd)

In [322]:
model = Sequential()

model.add(BatchNormalization(trainable=True))
model.add(Dense(2000, activation='linear'))
model.add(Dense(1000, activation = 'linear'))
model.add(Dropout(0.25))
model.add(Dense(1000, activation = 'relu'))

model.add(Dense(8, activation='softmax'))


In [327]:
model.compile(optimizer = Adam(0.0000001), loss = 'mse', metrics = ['accuracy'])
model.fit(xTrain, yTrainOrd, batch_size=256, epochs=2, validation_data=(xTest, yTestOrd))

Epoch 1/2
125/125 [==============================] - 1s 6ms/step - loss: 23.8785 - accuracy: 0.5249 - val_loss: 23.9735 - val_accuracy: 0.4484
Epoch 2/2
125/125 [==============================] - 1s 4ms/step - loss: 23.8786 - accuracy: 0.5258 - val_loss: 23.9735 - val_accuracy: 0.4512


In [169]:
predict = model.predict(xTest[0:10])

In [193]:
predict = model.predict(xTest[0:3])
print(list(map(int,1000*predict[1])))

[988, 0, 4, 4, 0, 0, 1, 0]


# Calculation of probabilities of events

In [328]:
# First get coefs for victory for 1st, 2nd and 3rd places

predict = model.predict(xTest).tolist()
prob11 = [predict[i].index(sorted(predict[i], reverse = False)[0]) for i in range(len(predict))] # favourite to come first (indices)
prob21 = [predict[i].index(sorted(predict[i], reverse = False)[1]) for i in range(len(predict))] # 2nd favourite to come first (indices)
prob31 = [predict[i].index(sorted(predict[i], reverse = False)[2]) for i in range(len(predict))] # -//-

In [329]:
pr11 = [predict[i][prob11[i]] for i in range(len(predict))] # prelim.prob. of favourite to come first (actual values)
pr21 = [predict[i][prob21[i]] for i in range(len(predict))] # -//-
pr31 = [predict[i][prob31[i]] for i in range(len(predict))] # -//-
print(pr11[:5])

[6.627649418078363e-05, 0.003736220533028245, 1.5925495517876698e-06, 6.317717975434789e-07, 0.00021406548330560327]


In [330]:
# recalculate the probability

for i in range(len(predict)):
  sum = 1/pr11[i] + 1/pr21[i] + 1/pr31[i] + 1/(1 - (pr11[i]+pr21[i]+pr31[i]))

  pr11[i] = (1/pr11[i])/sum
  pr21[i] = (1/pr21[i])/sum
  pr31[i] = (1/pr31[i])/sum

print(pr21[:5])

[0.17442572702695522, 0.2603731884539716, 0.03919523449505196, 0.0008537757151548195, 0.27569812542515254]


In [332]:
QuinellaPr = [pr11[i]*(pr21[i]/(1-pr11[i]+0.0001)) + pr21[i]*(pr11[i]/(1-pr21[i] + 0.0001)) for i in range(len(predict))]                                     # 1st and 2nd place 
ExactaPr = [pr11[i]*(pr21[i]/(1-pr11[i]+0.0001)) for i in range(len(predict))]                                                                        # 1st and 2nd place exactly
TrioPr = [pr11[i]*(pr21[i]/(1-pr11[i]+0.0001))*(pr31[i]/(1-pr11[i]-pr21[i]+0.0001)) + pr21[i]*(pr11[i]/(1-pr21[i]+0.0001))*(pr31[i]/(1-pr11[i]-pr21[i]+0.0001)) +\
pr31[i]*(pr11[i]/(1-pr31[i]+0.0001))*(pr21[i]/(1-pr11[i]-pr31[i]+0.0001)) + pr31[i]*(pr21[i]/(1-pr31[i]+0.0001))*(pr11[i]/(1-pr21[i]-pr31[i]+0.0001)) for i in range(len(predict))]   # 1st, 2nd and 3rd prlace
TrifectaPr = [pr11[i]*(pr21[i]/(1-pr11[i]+0.0001))*(pr31[i]/(1-pr11[i]-pr21[i]+0.0001)) for i in range(len(predict))]                                        # 1st, 2nd and 3rd place exactly

In [334]:
# now lets calculate expected profits
QuinellaExpProfit = []
ExactaExpProfit = []
TrioExpProfit = []
TrifectaExpProfit = []

for i in range(10):
  QuinellaExpProfit.append(QuinellaPr[i] * dfBets['Quinella_refund', 'Exacta_refund', 'Trio_refund', 'Trifecta_refund'][i][0] - 100)
  ExactaExpProfit.append(ExactaPr[i] * dfBets['Quinella_refund', 'Exacta_refund', 'Trio_refund', 'Trifecta_refund'][i][1] - 100)
  TrioExpProfit.append(TrioPr[i] * dfBets['Quinella_refund', 'Exacta_refund', 'Trio_refund', 'Trifecta_refund'][i][2] - 100)
  TrifectaExpProfit.append(TrifectaPr[i] * dfBets['Quinella_refund', 'Exacta_refund', 'Trio_refund', 'Trifecta_refund'][i][3] - 100)

In [336]:
print(statistics.mean(QuinellaExpProfit))
print(statistics.mean(ExactaExpProfit))
print(statistics.mean(TrioExpProfit))
print(statistics.mean(TrifectaExpProfit))

335.1717373363741
805.5852748869585
122.80972663020077
1431.8497705582047


In [337]:
# now we can address the predictCSV file

predictCSV.head()

,raceinfo_id,place_num,place_code,raceno,distance,situation,raceclass_code,weather_name3,popular,roadtemp,temp,humid,motorbikeno,player_code,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,rank_now,rank_last,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,last90_numall,last90_numwin,last90_numwinwin,last90_hightime,last90_highplace,last90_highplace_home,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
0,52674,2,0,1,1,1,3,1,2,14,9,39,1,1315,0,3.46,1,0.066,3.526,3.526,8,13,403,408,43.985,0,0,0,10,0.0,0.0,3.47,3.536,3.500,22,0,0,3.500,0,1,...,0.0,3.539,3.47,0.16,3.493,3.493,3.50,3.556,5,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.17,3.516,3.516,3.49,3.550,3,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.19,3.520,3.520
1,52674,2,0,1,1,1,3,1,2,14,9,39,2,2119,0,3.42,0,0.097,3.517,3.517,7,21,401,388,45.820,0,1,0,9,10.0,10.0,3.43,3.527,3.506,20,0,0,3.506,0,0,...,0.0,3.561,3.44,0.17,3.507,3.507,3.47,3.563,2,0,0,0,0,0,0.0,0.0,0.0,0.0,3.561,3.46,0.19,3.513,3.513,3.43,3.527,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.517,3.517
2,52674,2,0,1,1,1,3,1,2,14,9,39,3,2005,10,3.41,0,0.086,3.496,3.506,8,20,382,386,49.582,0,1,1,8,10.0,20.0,3.42,3.506,3.474,21,0,0,3.474,4,1,...,0.0,3.500,3.45,0.18,3.440,3.450,3.50,3.520,3,1,0,0,0,0,33.3,33.3,33.3,0.0,3.500,3.45,0.20,3.430,3.440,3.42,3.506,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.000,0.00,0.00,3.496,3.506
3,52674,2,0,1,1,1,3,1,2,14,9,39,4,1214,10,3.36,1,0.095,3.455,3.465,8,12,351,355,53.643,0,2,2,6,20.0,40.0,3.40,3.495,3.481,25,0,0,3.477,0,1,...,0.0,3.494,3.40,0.20,3.456,3.466,3.44,3.504,4,0,2,0,0,0,0.0,50.0,50.0,0.0,3.494,3.40,0.17,3.424,3.434,3.44,3.504,4,0,2,0,0,0,0.0,50.0,50.0,0.0,3.494,3.40,0.17,3.424,3.434
4,52674,2,0,1,1,1,3,1,2,14,9,39,5,2514,20,3.35,1,0.097,3.447,3.467,6,25,327,337,59.579,1,2,2,5,30.0,50.0,3.38,3.477,3.460,28,0,0,3.458,3,0,...,11.1,3.465,3.39,0.18,3.384,3.404,3.53,3.529,5,0,0,1,0,0,0.0,0.0,20.0,20.0,3.465,3.39,0.22,3.349,3.369,3.50,3.552,3,0,0,0,0,0,0.0,0.0,0.0,33.3,3.543,3.45,0.25,3.402,3.422


In [414]:
smth = pd.read_csv('/content/drive/MyDrive/AIU/Test task moto races/dataset_sample/predict.csv')
print(len(smth))

40362


In [384]:
rws0 = []
rws0idx = []
for i in range(0,len(smth)):
  if len(smth[smth['raceinfo_id']==smth.loc[i,'raceinfo_id']]) !=8:
    if smth.loc[i,'raceinfo_id'] == np.array(rws0).any():
      continue
    else:
      rws0.append(smth.loc[i,'raceinfo_id'])
      rws0idx.append(i)


In [ ]:
from collections import Counter

c = Counter(rws0)
list(c) # check indices and their amt in the list

In [382]:
# we will introduce rather weak participant to the sets with 7 bikers so he wouldnt affect results much
pd0layer = predictCSV[:1].copy()

In [383]:
pd0layer

,raceinfo_id,place_num,place_code,raceno,distance,situation,raceclass_code,weather_name3,popular,roadtemp,temp,humid,motorbikeno,player_code,handicap,traialtime,home,hensa,shisou_hensa,soutei_time,age,graduation,rank_now,rank_last,point,last10_num1th,last10_num2th,last10_num3th,last10_numother,last10_rate2th,last10_rate3th,last10_trialave,last10_raceave,last10_racehigh,last90_numall,last90_numwin,last90_numwinwin,last90_hightime,last90_highplace,last90_highplace_home,...,p2_Other_180_All_All_raceaccident,p2_Other_180_All_All_racehigh,p2_Other_180_All_All_traialhigh,p2_Other_180_All_All_stave,p2_Other_180_All_All_shisou_hensa,p2_Other_180_All_All_soutei_time,p2_Other_90_All_All_trialavg,p2_Other_90_All_All_raceave,p2_Other_90_All_All_numall,p2_Other_90_All_All_num1th,p2_Other_90_All_All_num2th,p2_Other_90_All_All_num3th,p2_Other_90_All_All_numwin,p2_Other_90_All_All_numwinwin,p2_Other_90_All_All_rate1th,p2_Other_90_All_All_rate2th,p2_Other_90_All_All_rate3th,p2_Other_90_All_All_raceaccident,p2_Other_90_All_All_racehigh,p2_Other_90_All_All_traialhigh,p2_Other_90_All_All_stave,p2_Other_90_All_All_shisou_hensa,p2_Other_90_All_All_soutei_time,p2_Other_90_All_Here_trialavg,p2_Other_90_All_Here_raceave,p2_Other_90_All_Here_numall,p2_Other_90_All_Here_num1th,p2_Other_90_All_Here_num2th,p2_Other_90_All_Here_num3th,p2_Other_90_All_Here_numwin,p2_Other_90_All_Here_numwinwin,p2_Other_90_All_Here_rate1th,p2_Other_90_All_Here_rate2th,p2_Other_90_All_Here_rate3th,p2_Other_90_All_Here_raceaccident,p2_Other_90_All_Here_racehigh,p2_Other_90_All_Here_traialhigh,p2_Other_90_All_Here_stave,p2_Other_90_All_Here_shisou_hensa,p2_Other_90_All_Here_soutei_time
0,52674,2,0,1,1,1,3,1,2,14,9,39,1,1315,0,3.46,1,0.066,3.526,3.526,8,13,403,408,43.985,0,0,0,10,0.0,0.0,3.47,3.536,3.5,22,0,0,3.5,0,1,...,0.0,3.539,3.47,0.16,3.493,3.493,3.5,3.556,5,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.17,3.516,3.516,3.49,3.55,3,0,0,0,0,0,0.0,0.0,0.0,0.0,3.539,3.47,0.19,3.52,3.52


In [406]:
idxsUpd = []
for i in range(len(rws0idx)-1):
  if rws0[i+1] > rws0[i]:
    idxsUpd.append(rws0idx[i])
idxsUpd.append(rws0idx[i+1])

In [ ]:
len(idxsUpd)
len(list(c))
c = Counter(rws0)
c

In [389]:
def insert_row(idx, df, df_insert):
    dfA = df.iloc[:idx, ]
    dfB = df.iloc[idx:, ]

    df = dfA.append(df_insert).append(dfB).reset_index(drop = True)

    return df

In [419]:
# inserting rows to predict DF

for i in range(len(idxsUpd)):
  predictCSV = insert_row(idxsUpd[i], predictCSV, pd0layer)

In [421]:
prepPredict = predictCSV.to_numpy(dtype = 'float32')
nPredict = []

tmpX = prepPredict[0].tolist()

for i in range(1,len(prepPredict)):
  if i%8!=0:
    tmpX.extend(prepPredict[i])
  else:
    nPredict.append(tmpX)
    tmpX = []

    tmpX.extend(prepPredict[i])

nPredict.append(tmpX)

nPredict = np.array(nPredict)
nPredict = scalerX.transform(nPredict)

In [423]:
results = model.predict(nPredict)
results.shape

In [444]:
results[0].argsort()

array([4, 3, 5, 7, 6, 2, 1, 0])

In [442]:
resultStr = []
for i in range(len(results)):
  line = '_'.join(list(map(str,results[i].argsort())))
  resultStr.append(line)


In [435]:
predict_hit_rate = pd.DataFrame(data = predictCSV['raceinfo_id'].unique(), columns = ['raceinfo_id'])

In [446]:
predict_hit_rate['xorder'] = resultStr

In [448]:
predict_hit_rate.to_csv('/content/drive/MyDrive/AIU/Test task moto races/predict_hit_rate.csv')

In [449]:
predict_recovery_rate = predict_hit_rate.copy()

In [450]:
predict_recovery_rate['Quinella'] = 0
predict_recovery_rate['Exacta'] = 0
predict_recovery_rate['Trio'] = 0
predict_recovery_rate['Trifecta'] = 10 # based on mean expected income values


In [452]:
predict_recovery_rate.head()

,raceinfo_id,xorder,Quinella,Exacta,Trio,Trifecta
0,52674,4_3_5_7_6_2_1_0,0,0,0,10
1,52675,4_5_3_6_7_2_0_1,0,0,0,10
2,52676,5_7_3_6_4_2_1_0,0,0,0,10
3,52677,6_7_5_4_2_0_1_3,0,0,0,10
4,52678,6_5_4_7_3_2_0_1,0,0,0,10


In [453]:
predict_recovery_rate.to_csv('/content/drive/MyDrive/AIU/Test task moto races/predict_recovery_rate.csv')